In [ ]:
!pip uninstall torch torchvision torchaudio transformers vllm -y
!pip cache purge
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu126
!pip install --upgrade transformers vllm datasets tqdm 
!pip install -U gptqmodel --no-build-isolation -v
!pip install optimum
!pip install --force-reinstall triton==3.2.0
!huggingface-cli login --token ***    # Read
# !huggingface-cli login --token ***    # Write

Found existing installation: torch 2.6.0+cu124
Uninstalling torch-2.6.0+cu124:
  Successfully uninstalled torch-2.6.0+cu124
Found existing installation: torchvision 0.21.0+cu124
Uninstalling torchvision-0.21.0+cu124:
  Successfully uninstalled torchvision-0.21.0+cu124
Found existing installation: torchaudio 2.6.0+cu124
Uninstalling torchaudio-2.6.0+cu124:
  Successfully uninstalled torchaudio-2.6.0+cu124
Found existing installation: transformers 4.51.3
Uninstalling transformers-4.51.3:
  Successfully uninstalled transformers-4.51.3
Files removed: 0
Looking in indexes: https://download.pytorch.org/whl/cu126
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 867.0/867.0 MB 2.1 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.1/393.1 MB 2.9 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 88.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 67.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import torch
import torch.nn as nn
from transformers import AutoTokenizer, AutoModelForCausalLM
from tqdm.auto import tqdm
from datasets import load_dataset
import random
import numpy as np
from vllm import LLM, SamplingParams
from gptqmodel import GPTQModel, QuantizeConfig
from vllm.config import CompilationConfig, CompilationLevel

#####################################################################
# === SPEC NOTICE ===
# Only "load model" and "generate" function selection can be modified.
# DO NOT change PPL calculation, timing, or throughput logic.
#####################################################################


# === (Optional) Define your own custom generate function. ===
# This is useful if you want full control over KV cache and generation steps.
# You can modify this function to suit your needs.
# By default, we use model.generate() for simplicity and general use.
def generate(model, input_ids, past_key_values, max_new_tokens):
    input_ids = input_ids.clone()
    with torch.no_grad():
        # Prefill
        outputs = model.prefill_forward(
            input_ids,
            past_key_values=past_key_values,
            position_ids=None,
            attention_mask=None,
            cache_position=None,
            logits_to_keep=1,
        )
        past_key_values = outputs.past_key_values
        next_token = torch.argmax(outputs.logits, dim=-1)
        input_ids = torch.cat([input_ids, next_token], dim=-1)

        # Token-by-token Decoding
        for _ in range(max_new_tokens):
            pos = input_ids.shape[1]
            cache_position = torch.arange(
                pos, pos + 1, device=input_ids.device, dtype=torch.long
            )

            outputs = model(
                next_token,
                past_key_values=past_key_values,
                position_ids=cache_position.unsqueeze(0),
                cache_position=cache_position,
            )
            logits = outputs.logits
            next_token = torch.argmax(logits, dim=-1)
            input_ids = torch.cat([input_ids, next_token], dim=-1)
            past_key_values = outputs.past_key_values

    return input_ids


def evaluate_ppl(model_name, device="cuda:0"):
    model = GPTQModel.load(model_name)
    tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

    model.eval()
    model.to(device)
    test_dataset = load_dataset("wikitext", "wikitext-2-raw-v1", split="test")

    test_enc = tokenizer("\n\n".join(test_dataset["text"]), return_tensors="pt")
    model.seqlen = 2048
    test_enc = test_enc.input_ids.to(device)

    nsamples = test_enc.numel() // model.seqlen
    nlls = []
    for i in tqdm(range(nsamples), desc="Evaluating..."):
        batch = test_enc[:, (i * model.seqlen) : ((i + 1) * model.seqlen)]

        with torch.no_grad():
            lm_logits = model(batch).logits

        shift_logits = lm_logits[:, :-1, :].contiguous().float()
        shift_labels = test_enc[:, (i * model.seqlen) : ((i + 1) * model.seqlen)][:, 1:]

        loss_fct = nn.CrossEntropyLoss()
        loss = loss_fct(
            shift_logits.view(-1, shift_logits.size(-1)), shift_labels.view(-1)
        )
        neg_log_likelihood = loss.float() * model.seqlen
        nlls.append(neg_log_likelihood)

    ppl = torch.exp(torch.stack(nlls).sum() / (nsamples * model.seqlen))

    del model

    return ppl.item()


def main():
    ############## Set Up ##############
    torch.manual_seed(0)
    random.seed(0)

    max_new_tokens = 256  # Number of new tokens to generate
    device = "cuda:0"

    ### === TODO: Load your model (you may change this part) ===
    model_name = "zbyzby/Llama3.2-3B-Instruct-quantized"
    torch.cuda.empty_cache()
    vllm_model = LLM(
        model=model_name,
        kv_cache_dtype="auto",  # only supported in Amper GPU
        trust_remote_code=True,
        gpu_memory_utilization=0.9,
        max_num_seqs=64,
        max_model_len=2048,
        enable_chunked_prefill=True,
        enable_prefix_caching=True,
        tensor_parallel_size=1,
        speculative_config={
            "model": "zbyzby/Llama-3.2-1B-Instruct-GPTQ-Quant",
            "draft_tensor_parallel_size": 1,
            "num_speculative_tokens": 5,
        },
        compilation_config={
            "backend": "inductor",
            "custom_ops": ["vllm_ops"],
            "mode": "reduce-overhead",
            "enable_cudagraphs": True,
            "max_autotune": True,
            "level": CompilationLevel.PIECEWISE,
        },
    )
    sampling_params = SamplingParams(
        temperature=0.0, max_tokens=max_new_tokens, top_p=1.0
    )
    #####################################

    # === (Optional) Uncomment the following lines if using the custom generate() function. ===
    # model.prefill_forward = model.forward

    warmup_prompt = "Explain what AI is."
    # inputs = tokenizer(warmup_prompt, return_tensors="pt").to(device)
    # input_ids = inputs["input_ids"]
    # attention_mask = inputs["attention_mask"]

    # === (Optional) Set up StaticCache for manual KV cache management ===
    # from transformers import StaticCache
    # past_key_values = StaticCache(
    #     config=model.config,
    #     max_batch_size=1,
    #     max_cache_len=max_new_tokens + 16,
    #     device=model.device,
    #     dtype=torch.float16
    # )
    ####################################################################

    for i in tqdm(range(5), desc="Warm Up..."):
        #  === Default: use model.generate() for end-to-end warm-up ===
        _ = vllm_model.generate([warmup_prompt], sampling_params)

        # === (Optional) Use custom generate() if uncommented ===
        # generated = generate(model, input_ids, past_key_values, max_new_tokens)
        # past_key_values.reset()

    prompt = "How to learn a new language?"
    # inputs = tokenizer(prompt, return_tensors="pt").to(device)
    # input_ids = inputs["input_ids"]
    # attention_mask = inputs["attention_mask"]
    tputs = []
    time_record = []
    for _ in tqdm(range(10), desc="Test Inference"):
        torch.cuda.synchronize()
        start = torch.cuda.Event(enable_timing=True)
        end = torch.cuda.Event(enable_timing=True)
        start.record()

        # === Default: Use model.generate() for end-to-end timing ===
        generated = vllm_model.generate([prompt], sampling_params)

        # === Optional: Use custom generate() if uncommented ===
        # generated = generate(model, input_ids, past_key_values, max_new_tokens)
        # past_key_values.reset()

        end.record()
        torch.cuda.synchronize()
        elapsed_ms = start.elapsed_time(end)
        tput = max_new_tokens / (elapsed_ms / 1000)
        time_record.append(elapsed_ms / 1000)
        tputs.append(tput)

    response = generated[0].outputs[0].text
    sorted_tputs = np.sort(tputs)[2:-2]
    org_tput = np.mean(sorted_tputs)
    print(f"Prompt: {prompt}\nResponse: {response}\n")

    print(f"Time Record: {time_record}")
    print(f"Throughput Record: {tputs} toks/s\n")

    del vllm_model

    ### Your final throughput result ###
    print(f"Throughput: {org_tput} toks/s")

    torch.cuda.empty_cache()
    ppl = evaluate_ppl(model_name, device)
    print(f"Perplexity (PPL): {ppl}")

    # Save results to CSV
    import csv

    rounded_tput = round(org_tput, 1)
    ppl = round(ppl, 2)

    with open("result.csv", mode="w", newline="") as file:
        writer = csv.writer(file)
        writer.writerow(["Id", "value"])
        writer.writerow([0, ppl])
        writer.writerow([1, rounded_tput])


if __name__ == "__main__":
    main()


2025-06-01 13:37:03.984373: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748785024.393669      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748785024.507283      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


INFO 06-01 13:37:18 [__init__.py:243] Automatically detected platform cuda.

INFO  ENV: Auto setting PYTORCH_CUDA_ALLOC_CONF='expandable_segments:True' for memory saving.      
INFO  ENV: Auto setting CUDA_DEVICE_ORDER=PCI_BUS_ID for correctness.                              
INFO 06-01 13:37:25 [__init__.py:31] Available plugins for group vllm.general_plugins:
INFO 06-01 13:37:25 [__init__.py:33] - lora_filesystem_resolver -> vllm.plugins.lora_resolvers.filesystem_resolver:register_filesystem_resolver
INFO 06-01 13:37:25 [__init__.py:36] All plugins in this group will be loaded. Set `VLLM_PLUGINS` to control which plugins to load.


config.json:   0%|          | 0.00/1.37k [00:00<?, ?B/s]

WARNING 06-01 13:37:26 [config.py:3096] Your Tesla T4 device (with compute capability 7.5) doesn't support torch.bfloat16. Falling back to torch.float16 for compatibility.
WARNING 06-01 13:37:26 [config.py:3135] Casting torch.bfloat16 to torch.float16.
INFO 06-01 13:37:42 [config.py:793] This model supports multiple tasks: {'generate', 'reward', 'embed', 'score', 'classify'}. Defaulting to 'generate'.
WARNING 06-01 13:37:43 [config.py:907] gptq quantization is not fully optimized yet. The speed can be slower than non-quantized models.
WARNING 06-01 13:37:43 [arg_utils.py:1583] Compute Capability < 8.0 is not supported by the V1 Engine. Falling back to V0. 
INFO 06-01 13:37:43 [config.py:2118] Chunked prefill is enabled with max_num_batched_tokens=2048.
INFO 06-01 13:37:43 [llm_engine.py:230] Initializing a V0 LLM engine (v0.9.0.1) with config: model='zbyzby/Llama3.2-3B-Instruct-quantized', speculative_config=None, tokenizer='zbyzby/Llama3.2-3B-Instruct-quantized', skip_tokenizer_init=F

tokenizer_config.json:   0%|          | 0.00/50.5k [00:00<?, ?B/s]

...

Generating train split:   0%|          | 0/36718 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3760 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (289077 > 131072). Running this sequence through the model will result in indexing errors


Evaluating...:   0%|          | 0/141 [00:00<?, ?it/s]

Perplexity (PPL): 11.217229843139648
